In [1]:
from imprint.nb_util import setup_nb
setup_nb()
import asyncio
import redis
import numpy as np

import confirm.cloud.clickhouse as ch

from confirm.adagrid.validate import AdaValidate
from confirm.adagrid.calibrate import AdaCalibrate
from confirm.adagrid.validate import ada_validate
from confirm.adagrid.calibrate import ada_calibrate
from confirm.cloud.redis_heartbeat import HeartbeatThread
import confirm.adagrid.adagrid as adagrid
import imprint as ip
from imprint.models.ztest import ZTest1D

import clickhouse_connect
clickhouse_connect.common.set_setting('autogenerate_session_id', False)

### Glossary

- **coordination**: All workers come together and divide up all tiles.
- **step**: A worker checks the convergence criterion, select tiles, and simulates.
- **packet**: The unit of simulation work that a worker requests from the DB.
- **batch**: The unit of simulation work that is passed to a Model.

In [11]:
ch.clear_dbs(drop_all_redis_keys=True)

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None


No Clickhouse databases to drop.
drop_all_redis_keys=True, deleting redis keys [b'6d86af8ae6c941fc8824929622286e69:coordination_id', b'6d86af8ae6c941fc8824929622286e69:next_worker_id', b'6d86af8ae6c941fc8824929622286e69:worker_2:step_0:n_packets', b'6d86af8ae6c941fc8824929622286e69:worker_2:step_0:n_tiles', b'6d86af8ae6c941fc8824929622286e69:worker_2:step_0:packet_0:insert', b'6d86af8ae6c941fc8824929622286e69:worker_2:step_1:n_packets', b'6d86af8ae6c941fc8824929622286e69:worker_2:step_1:n_tiles', b'6d86af8ae6c941fc8824929622286e69:worker_2:step_1:packet_0:insert', b'6d86af8ae6c941fc8824929622286e69:worker_2:step_1:packet_1:insert', b'76272da01043472fae5e67093e7c40e0:coordination_id', b'76272da01043472fae5e67093e7c40e0:next_worker_id', b'76272da01043472fae5e67093e7c40e0:worker_2:step_0:n_packets', b'76272da01043472fae5e67093e7c40e0:worker_2:step_0:n_tiles', b'7ca46162813c4d56bfdc02cf926c9fc2:coordination_id', b'7ca46162813c4d56bfdc02cf926c9fc2:next_worker_id', b'7ca46162813c4d56bfdc02cf

In [7]:
db = ch.Clickhouse.connect(job_id="8603b7cfab1445bc9a3ebfcfcd956255")

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/8603b7cfab1445bc9a3ebfcfcd956255
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Connected to job 8603b7cfab1445bc9a3ebfcfcd956255


In [8]:
tiles = db.get_tiles()

In [9]:
tiles

,id,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,active
0,262144,0,-0.50000,0.50000,1,8192,0,0,2,0,2,1,0
1,137439477760,262144,-0.75000,0.25000,1,8192,0,1,2,0,2,3,0
2,137439477761,262144,-0.25000,0.25000,1,8192,0,1,2,1,2,3,0
3,274878431232,137439477761,-0.37500,0.12500,1,8192,0,2,2,0,2,5,0
4,274878431233,137439477761,-0.12500,0.12500,1,8192,0,2,2,1,2,5,0
5,274878431234,137439477760,-0.87500,0.12500,1,8192,0,2,2,2,2,5,1
6,274878431235,137439477760,-0.62500,0.12500,1,8192,0,2,2,3,2,5,0
7,412317384704,274878431233,-0.18750,0.06250,1,8192,0,3,2,0,2,7,0
8,412317384705,274878431233,-0.06250,0.06250,1,8192,0,3,2,1,2,7,0
9,412317384706,274878431232,-0.43750,0.06250,1,8192,0,3,2,2,2,7,1


In [6]:
db.get_work(0, 2, 2, 3)

,id,active,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time
0,274878431235,1,137439477760,-0.625,0.125,1,8192,0,2,2,3,2,5


In [2]:
import time
start = time.time()
db = ch.Clickhouse.connect()
print('1', time.time() - start)
g = ip.cartesian_grid(
    theta_min=[-1], theta_max=[1], n=[50000], null_hypos=[ip.hypo("x0 < 0")]
)
print('2', time.time() - start)

import inspect
sig = inspect.signature(ada_validate)
kwargs={k: v.default for k,v in sig.parameters.items()}
kwargs.update(dict(lam=-1.96, prod=False))
n_workers=2
ada = adagrid.Adagrid()
print('3', time.time() - start)
await ada.init(ZTest1D, g, db, AdaValidate, print, None, n_workers, kwargs, worker_id = 1)
print('4', time.time() - start)

INFO:confirm.cloud.clickhouse:[worker_id=<ContextVar name='worker_id' default=None at 0x28c40ef20>] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=<ContextVar name='worker_id' default=None at 0x28c40ef20>] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/793bd9f8305b493b9cda375cc31ac8fb
INFO:confirm.cloud.clickhouse:[worker_id=<ContextVar name='worker_id' default=None at 0x28c40ef20>] 
Connected to job 793bd9f8305b493b9cda375cc31ac8fb
DEBUG:imprint.grid:[worker_id=<ContextVar name='worker_id' default=None at 0x28c40ef20>] 
_gen_short_uuids(n=50000, worker_id=1, t=1676339470, n_bits=18, worker_bits=18) = [4516706768046522368 4516706768046522369 4516706768046522370, ...]:


1 0.3820929527282715
2 0.40077900886535645
3 0.4009571075439453


DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Initialized database with 25000 tiles and 1 null hypos. The tiles are split between 2 workers with packet_size=1024.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Initial worker ids: [1, 2]
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set_or_append(config) -> False None
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set(config) -> False config
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
writing 25000 tiles
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set(null_hypos) -> False null_hypos


4 2.9688799381256104


In [8]:
ada.db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,active
0,4516572696347410432,0,-0.99998,0.00002,1,8192,0,0,1,0,1,1.676338e+09,1
1,4516572696347410433,0,-0.99994,0.00002,1,8192,0,0,1,0,1,1.676338e+09,1
2,4516572696347410434,0,-0.99990,0.00002,1,8192,0,0,1,0,1,1.676338e+09,1
3,4516572696347410435,0,-0.99986,0.00002,1,8192,0,0,1,0,1,1.676338e+09,1
4,4516572696347410436,0,-0.99982,0.00002,1,8192,0,0,1,0,1,1.676338e+09,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,4516572696347435427,0,-0.00018,0.00002,1,8192,0,0,2,12,1,1.676338e+09,1
24996,4516572696347435428,0,-0.00014,0.00002,1,8192,0,0,2,12,1,1.676338e+09,1
24997,4516572696347435429,0,-0.00010,0.00002,1,8192,0,0,2,12,1,1.676338e+09,1
24998,4516572696347435430,0,-0.00006,0.00002,1,8192,0,0,2,12,1,1.676338e+09,1


In [10]:
ada.db.get_results()

,id,parent_id,theta0,radii0,null_truth0,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,processor_id,processing_time,grid_cost,sim_cost,total_cost,total_cost_order,tie_bound_order,tie_sum,tie_est,tie_cp_bound,tie_bound,K,active,eligible
0,4516572696347422845,0,-0.50346,0.00002,1,0,0,1,12,1,1.676338e+09,1,1.676338e+09,5.762736e-07,0.002451,0.002451,-0.0,-0.009409,57,0.006958,0.009409,0.009409,8192,False,False
1,4516572696347422846,0,-0.50342,0.00002,1,0,0,1,12,1,1.676338e+09,1,1.676338e+09,5.762736e-07,0.002451,0.002451,-0.0,-0.009409,57,0.006958,0.009409,0.009409,8192,False,False
2,4516572696347422847,0,-0.50338,0.00002,1,0,0,1,12,1,1.676338e+09,1,1.676338e+09,5.762736e-07,0.002451,0.002451,-0.0,-0.009409,57,0.006958,0.009409,0.009409,8192,False,False
3,4516572696347422848,0,-0.50334,0.00002,1,0,0,1,12,1,1.676338e+09,1,1.676338e+09,5.762736e-07,0.002451,0.002451,-0.0,-0.009409,57,0.006958,0.009409,0.009409,8192,False,False
4,4516572696347422849,0,-0.50330,0.00002,1,0,0,1,12,1,1.676338e+09,1,1.676338e+09,5.762736e-07,0.002451,0.002451,-0.0,-0.009409,57,0.006958,0.009409,0.009409,8192,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14543,4516576132321248251,4516572696347420911,-0.58082,0.00002,1,0,2,1,0,1,1.676338e+09,1,1.676338e+09,4.566536e-07,0.001528,0.001529,-0.0,-0.000000,94,0.005737,0.007266,0.007266,16384,True,True
14544,4516576132321248252,4516572696347420912,-0.58078,0.00002,1,0,2,1,0,1,1.676338e+09,1,1.676338e+09,4.566536e-07,0.001528,0.001529,-0.0,-0.000000,94,0.005737,0.007266,0.007266,16384,True,True
14545,4516576132321248253,4516572696347420913,-0.58074,0.00002,1,0,2,1,0,1,1.676338e+09,1,1.676338e+09,4.566536e-07,0.001528,0.001529,-0.0,-0.000000,94,0.005737,0.007266,0.007266,16384,True,True
14546,4516576132321248254,4516572696347420914,-0.58070,0.00002,1,0,2,1,0,1,1.676338e+09,1,1.676338e+09,4.566536e-07,0.001528,0.001529,-0.0,-0.000000,94,0.005737,0.007266,0.007266,16384,True,True


In [18]:
ada.db.get_work(0, 1, 3, 0)

,id,active,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time
0,4516576269760200704,1,4516572696347420916,-0.58062,0.00002,1,16384,0,3,1,0,1,1.676338e+09
1,4516576269760200705,1,4516572696347420917,-0.58058,0.00002,1,16384,0,3,1,0,1,1.676338e+09
2,4516576269760200706,1,4516572696347420918,-0.58054,0.00002,1,16384,0,3,1,0,1,1.676338e+09
3,4516576269760200707,1,4516572696347420919,-0.58050,0.00002,1,16384,0,3,1,0,1,1.676338e+09
4,4516576269760200708,1,4516572696347420920,-0.58046,0.00002,1,16384,0,3,1,0,1,1.676338e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,4516576269760201723,1,4516572696347420085,-0.61386,0.00002,1,16384,0,3,1,0,1,1.676338e+09
1020,4516576269760201724,1,4516572696347420086,-0.61382,0.00002,1,16384,0,3,1,0,1,1.676338e+09
1021,4516576269760201725,1,4516572696347420087,-0.61378,0.00002,1,16384,0,3,1,0,1,1.676338e+09
1022,4516576269760201726,1,4516572696347420088,-0.61374,0.00002,1,16384,0,3,1,0,1,1.676338e+09


In [17]:
df = ada.db.get_results()
np.sum(df['step_id'] == 3)

0

In [12]:
ada.db.n_processed_tiles(1, 3)

0

In [12]:
async with ada.db.heartbeat(ada.worker_id):
    await ada._run_local()

DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Acquired lock 793bd9f8305b493b9cda375cc31ac8fb:heartbeat:1
DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Added worker 1 to workers set.
DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Extended lock 793bd9f8305b493b9cda375cc31ac8fb:heartbeat:1 for 30 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Skipping packet. Flag 793bd9f8305b493b9cda375cc31ac8fb:worker_1:step_4:packet_0:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Empty packet indicates that we might be done with work for this step.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Done with work for this step.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Convergence!!
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Waiting for coordination 1
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Leading the coordination
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Waiting workers: [1]
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Registered workers: [1]
D

In [8]:
async with HeartbeatThread(ada.db.redis_con, ada.db.job_id, ada.worker_id):
    await ada.process_step(0, 0)

DEBUG:confirm.adagrid:[worker_id=None] 
Acquired lock dcfe4aff65e84386af721900203e0317:heartbeat:1
DEBUG:confirm.adagrid:[worker_id=None] 
Added worker 1 to workers set.
DEBUG:confirm.adagrid:[worker_id=None] 
Extended lock dcfe4aff65e84386af721900203e0317:heartbeat:1 for 30 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_0:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_1:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_2:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_3:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84

In [ ]:
db.get_reports()

,worker_id,step_id,packet_id,n_tiles,runtime_get_work,runtime_process_tiles,runtime_insert_results,status
4,1,0,0,1024,0.089789,0.064014,0.000085,WORKING
3,1,0,1,1024,0.086273,0.018211,0.000090,WORKING
0,1,0,2,1024,0.114576,0.013122,0.000064,WORKING
5,1,0,3,1024,0.026534,0.014037,0.000126,WORKING
2,1,0,4,1024,0.046059,0.012358,0.000127,WORKING
1,1,0,5,1024,0.177392,0.023063,0.000092,WORKING
6,1,0,6,1024,0.041068,0.015855,0.000069,WORKING
8,1,0,7,1024,0.050865,0.014689,0.000110,WORKING
9,1,0,8,1024,0.132194,0.016320,0.000176,WORKING
7,1,0,9,1024,0.033667,0.016473,0.000112,WORKING


In [ ]:
await ada.new_step(1)

INFO:confirm.adagrid.validate:[worker_id=None] 
Preparing new step with 1024 parent tiles.
DEBUG:confirm.cloud.clickhouse:[worker_id=None] 
finish:                     id  step_id  packet_id  active  finisher_id  refine  \
0  4515240363132477857        0         24   False            1   False   
1  4515240363132477858        0         24   False            1   False   
2  4515240363132477859        0         24   False            1   False   
3  4515240363132477860        0         24   False            1   False   
4  4515240363132477861        0         24   False            1   False   

   deepen  split  
0    True  False  
1    True  False  
2    True  False  
3    True  False  
4    True  False  
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=1024, worker_id=1, t=1676318137, n_bits=18, worker_bits=18) = [4515240775449313280 4515240775449313281 4515240775449313282, ...]:
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=0, worker_id=1, t=1676318138, n_bits=18, work

<WorkerStatus.NEW_STEP: 5>

In [11]:
# TODO: test running coordinate with a single work and confirming that it
# updates eligible and active.
pre = ch._query_df(db.client, 'select * from results')
pre['active'].sum(), pre['eligible'].sum()

(25000, 25000)

In [25]:
async with HeartbeatThread(ada.db.redis_con, ada.db.job_id, ada.worker_id):
    status = await ada.coordinate()

In [20]:
ch._query_df(db.client, 'select * from results')['active'].sum()

25000

In [23]:
ch._query_df(db.client, 'select * from results where coordination_id = 1')

,id,active,parent_id,theta0,radii0,null_truth0,step_id,creator_id,creation_time,coordination_id,worker_id,packet_id,processor_id,processing_time,eligible,grid_cost,sim_cost,total_cost,total_cost_order,tie_bound_order,tie_sum,tie_est,tie_cp_bound,tie_bound,K


In [12]:
ada.db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,K,step_id,creator_id,creation_time,worker_id,packet_id,active
0,4514895185200807936,0,-0.99998,0.00002,1,8192,0,1,1.676313e+09,1,0,1
1,4514895185200807937,0,-0.99994,0.00002,1,8192,0,1,1.676313e+09,1,0,1
2,4514895185200807938,0,-0.99990,0.00002,1,8192,0,1,1.676313e+09,1,0,1
3,4514895185200807939,0,-0.99986,0.00002,1,8192,0,1,1.676313e+09,1,0,1
4,4514895185200807940,0,-0.99982,0.00002,1,8192,0,1,1.676313e+09,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26019,4514895597517669371,4514895185200831859,-0.04306,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26020,4514895597517669372,4514895185200831860,-0.04302,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26021,4514895597517669373,4514895185200831861,-0.04298,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26022,4514895597517669374,4514895185200831862,-0.04294,0.00002,1,16384,1,1,1.676313e+09,1,0,1


In [10]:
await ada.process_step(1)

DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Simulated 1024 tiles in 0.07 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
No more work for this step.
DEBUG:confirm.cloud.clickhouse:[worker_id=None] 
writing 1024 results
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
inserted packet results for (step_id = 1, packet_id=0) with 1024 results
